# 초급 튜토리얼: MobileNet 이미지 분류 모델 컴파일

# DEEPX DX-COM 튜토리얼 (Ubuntu 24.04 + JupyterLab)
이 노트북은 DEEPX **DX-COM v2.0.0**을 사용하여 ONNX 모델을 **.dxnn**으로 컴파일하는 실습을 단계별로 안내합니다.
**중요:** DX-COM 패키지(tar.gz)를 이미 이 작업 디렉터리에 내려받았다고 가정합니다.
압축을 풀면 `./dx_com/` 폴더가 생기며, 그 안에 `dx_com` 실행파일과 샘플이 포함되어 있습니다.

**학습 목표**
- PyTorch → ONNX 내보내기
- JSON 구성 작성 (입력/전처리/캘리브레이션)
- DX-COM으로 .dxnn 생성 및 확인


## 1) 의존 패키지 설치 및 DX-COM 패키지 해제

In [ ]:
# 시스템 라이브러리 (OpenCV 의존) 및 빌드 도구
!sudo apt-get update -y && sudo apt-get install -y --no-install-recommends libgl1-mesa-glx libglib2.0-0 make

# DX-COM 패키지(tar.gz)가 현재 폴더에 있다고 가정하고 해제
# 예) 파일명이 dx_com_M1_v2.0.0.tar.gz 라면 아래 줄의 파일명을 맞춰주세요.
!ls -1 *.tar.gz || true
!tar -xzf dx_com_M1_v2.0.0.tar.gz || echo '압축 파일명이 다르면 위 명령을 수정하세요.'

# 파이토치/토치비전 설치
!pip install --quiet torch torchvision onnx

## 2) MobileNetV2 파이토치 모델을 ONNX로 내보내기

In [ ]:
import torch, torchvision

model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()
dummy = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"
torch.onnx.export(model, dummy, onnx_path, input_names=["input"], output_names=["output"], opset_version=13)
print("✅ ONNX 저장:", onnx_path)

## 3) DX-COM용 JSON 구성 파일 생성 (전처리 + 캘리브레이션)

In [ ]:
import json, os

# 캘리브레이션 이미지 폴더 예시: ./dx_com/calibration_dataset
cfg = {
  "inputs": {"input": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./dx_com/calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}
with open("MobilenetV2.json","w") as f:
    json.dump(cfg, f, indent=2)
print("✅ JSON 저장: MobilenetV2.json")
os.system("sed -n '1,120p' MobilenetV2.json")

## 4) DX-COM으로 컴파일 실행 (.dxnn 생성)

In [ ]:
# 출력 폴더 지정
!mkdir -p dx_com/output/mobilenetv2

# DX-COM 실행 (모델/구성/출력 지정)
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o dx_com/output/mobilenetv2 || echo 'DX-COM 실행 실패시 경로/파일명 확인'

# 결과 확인
!ls -lh dx_com/output/mobilenetv2

## 5) 트러블슈팅 체크리스트
- `Permission denied`: `chmod +x dx_com/dx_com`
- 입력명/shape 불일치: JSON `inputs`와 ONNX 입력명이 일치하는지 확인
- 정확도 저하: `calibration_num` 증가, `calibration_method`(ema/minmax) 비교
